In [1]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import cv2
from skimage import measure, segmentation, feature
from vis_utils import load_volume, VolumeVisualizer, ColorMapVisualizer
from scipy.ndimage import zoom
from skimage.morphology import skeletonize, skeletonize_3d, binary_dilation

from skimage import filters, morphology

from scipy.ndimage.filters import convolve, correlate
from scipy import signal

from skimage.filters import frangi, sato
from skimage.draw import line_nd

from PIL import Image
import pickle

In [2]:
source_dir = './data/P12/'
skeleton = np.load(source_dir + 'skeleton.npy')
thiccness_map = np.load(source_dir + 'thiccness-map.npy')

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
skeleton = np.pad(skeleton, 1)
thiccness_map = np.pad(thiccness_map, 1)

In [3]:
def visualize_addition(base, base_with_addition):
    base = (base.copy() > 0).astype(np.uint8)
    addition = (base_with_addition > 0).astype(np.uint8)
    addition[base == 1] = 0
    ColorMapVisualizer(base + addition * 4).visualize()
    
def visualize_lsd(lsd_mask):
    ColorMapVisualizer(lsd_mask.astype(np.uint8)).visualize()
    
def visualize_gradient(lsd_mask):
    ColorMapVisualizer(lsd_mask.astype(np.uint8)).visualize(gradient=True)
    
def visualize_mask_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8), binary=True).visualize()
    
def visualize_mask_non_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8) * 255, binary=False).visualize()
    
def visualize_skeleton(mask, visualize_mask=True, visualize_both_versions=False):
    skeleton = skeletonize((mask > 0).astype(np.uint8))
    if not visualize_mask or visualize_both_versions:
        VolumeVisualizer(skeleton, binary=True).visualize()
    if visualize_mask or visualize_both_versions:
        skeleton = skeleton.astype(np.uint8) * 4
        mask = (mask > 0).astype(np.uint8) * 3
        mask[skeleton != 0] = 0
        ColorMapVisualizer(skeleton + mask).visualize()

def visualize_ultimate(lsd, base_mask):
    visualize_lsd(lsd)
    visualize_mask_non_bin(lsd)
    visualize_addition(base_mask, lsd)
    visualize_skeleton(lsd, visualize_mask=True)

In [4]:
def trim_skeleton(skeleton):   
    # TODO: fix wczesniejszych skeletonow !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!TODOTODOTODOTODOTODOTODOTODOTODOTODOTODOTODOTO
    padded_skeleton = np.pad(skeleton, 1)
    new_skeleton = np.zeros(padded_skeleton.shape)
    queue = [tuple(np.argwhere(padded_skeleton)[0])]
    
    while(len(queue) > 0):
        x, y, z = queue.pop(0)
        
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                for dz in [-1, 0, 1]:
                    if dx == dy == dz == 0:
                        continue
                    
                    neighbour_x = x + dx
                    neighbour_y = y + dy
                    neighbour_z = z + dz
                    if padded_skeleton[neighbour_x, neighbour_y, neighbour_z] == 0:
                        continue
                        
                    if new_skeleton[neighbour_x, neighbour_y, neighbour_z] == 0:
                        queue.append((neighbour_x, neighbour_y, neighbour_z))
                        new_skeleton[neighbour_x, neighbour_y, neighbour_z] += 1;
                        new_skeleton[x, y, z] += 1
                        
    return (new_skeleton[1:-1, 1:-1, 1:-1] > 1).astype(np.uint8)



def mark_bifurcation_regions(skeleton):
    
    padded_skeleton = np.pad(skeleton, 1)
    bifurcations_map = np.zeros(padded_skeleton.shape)
    
    for skeleton_voxel in np.argwhere(padded_skeleton > 0):
        x, y, z = tuple(skeleton_voxel)
        kernel_radius = 1
        kernel = np.ones((3, 3, 3))
        kernel[1, 1, 1] = 0
        
        skeleton_slice = padded_skeleton[
            x-kernel_radius:x+kernel_radius + 1,
            y-kernel_radius:y+kernel_radius + 1,
            z-kernel_radius:z+kernel_radius + 1
        ]
        
        intersections = (skeleton_slice > 0) * kernel
        bifurcations_map[x, y, z] = np.sum(intersections)
        
    return (bifurcations_map[1:-1, 1:-1, 1:-1] > 2).astype(np.uint8)

def mark_leaves(skeleton):
    trimmed = trim_skeleton(skeleton)
    leaves = skeleton - trimmed
    return leaves

def mark_nodes(skeleton):
    bifurcation_map = mark_bifurcation_regions(skeleton)
    leaves_map = mark_leaves(skeleton)
    return bifurcation_map + leaves_map

In [67]:
class Node:
    def __init__(self, coords):
        self.coords = coords
        self.edges = []
        self.data = {}
            
    def add_edge(self, edge):
        self.edges.append(edge)
        
    def get_neighbours(self):
        return [e.node_a if e.node_a != self else e.node_b for e in self.edges]
    
    def copy_without_edges(self):
        copied_node = Node(self.coords)
        copied_node.data = self.data
        return copied_node
    
    def __setitem__(self, key, value):
        self.data[key] = value
    
    def __getitem__(self, key):
        return self.data[key]
    
    def __hash__(self):
        return hash(self.coords)
    
    def __repr__(self):
        return f'Node {str(self.coords)}'
        
class Edge:
    def __init__(self, node_a, node_b):
        self.node_a = node_a
        self.node_b = node_b
        self.data = {}
        
    def __setitem__(self, key, value):
        self.data[key] = value
    
    def __getitem__(self, key):
        return self.data[key]
    
    def __repr__(self):
        return f'Edge {self.node_a.coords} -> {self.node_b.coords}'

In [69]:
def construct_graph(skeleton, nodes_mask, thiccness_map):
    nodes_labels = measure.label(nodes_mask)
    nodes_props = measure.regionprops(nodes_labels)
    print('nodes found (regions on nodes mask):', nodes_labels.max())
    voxel_to_node = dict()
    
    for props in nodes_props:
        if props.label < 1:
            continue
            
        node = Node(tuple(props.coords[0]))
        node['voxels'] = props.coords
        node['label'] = props.label
        node['thiccness'] = thiccness_map[tuple(props.coords[0])]
        
        for c in props.coords:
            voxel_to_node[tuple(c)] = node
            
    edges_mask = skeleton - nodes_mask
    edges_labels = measure.label(edges_mask > 0)
    print('edges found:', edges_labels.max())
    
    visited = np.zeros(skeleton.shape, dtype=np.bool)
    
    def find_touching_nodes(source_voxel):
        touching_nodes = set()
        queue = [source_voxel]
        while len(queue) > 0:
            x, y, z = queue.pop(0)
            
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    for dz in [-1, 0, 1]:
                        if dx == dy == dz == 0:
                            continue

                        neighbour_x = x + dx
                        neighbour_y = y + dy
                        neighbour_z = z + dz
                        if visited[neighbour_x, neighbour_y, neighbour_z]:
                            continue
                            
                        potential_node = voxel_to_node.get((neighbour_x, neighbour_y, neighbour_z))
                        if potential_node is not None:
                            touching_nodes.add(potential_node)

                        if edges_mask[neighbour_x, neighbour_y, neighbour_z] == 1:
                            queue.append((neighbour_x, neighbour_y, neighbour_z))
                            visited[neighbour_x, neighbour_y, neighbour_z] = True
        return list(touching_nodes)
       
        
    edges_props = measure.regionprops(edges_labels)
    
    edges = []
    bad_edges = []
    for props in edges_props:
        edge_voxel = props.coords[0]
        touching_nodes = find_touching_nodes(edge_voxel)
        if len(touching_nodes) != 2:
            print(f'bad edge found! touching nodes count: {len(touching_nodes)}')
            bad_edges.append(props.coords)
            continue
            
        edge = Edge(touching_nodes[0], touching_nodes[1])
        edge['voxels'] = props.coords
        edges.append(edge)
    return edges, bad_edges

In [70]:
bifurcations_mask = mark_bifurcation_regions(skeleton)
leaves_mask = mark_leaves(skeleton)
nodes_mask = ((bifurcations_mask + leaves_mask) > 0).astype(np.uint8)

In [71]:
%%time
edges, bad_edges = construct_graph(skeleton, nodes_mask, thiccness_map)
print("Number of bad edged found:", len(bad_edges))

nodes found (regions on nodes mask): 2300
edges found: 2338
bad edge found! touching nodes count: 1
bad edge found! touching nodes count: 1
bad edge found! touching nodes count: 1
bad edge found! touching nodes count: 1
bad edge found! touching nodes count: 1
bad edge found! touching nodes count: 1
bad edge found! touching nodes count: 1
Number of bad edged found: 7
CPU times: user 5.96 s, sys: 424 ms, total: 6.39 s
Wall time: 6.36 s


In [72]:
skeleton_without_bad_edges = skeleton.copy()
for bad_edge in bad_edges:
    for coords in bad_edge:
        skeleton_without_bad_edges[tuple(coords)] = 0
# visualize_addition(skeleton_without_bad_edges, skeleton)

## Graph purification

In [78]:
def remove_redundant_edges(edges):
    def universal_hash(node_a, node_b):
        hash_a = hash(node_a)
        hash_b = hash(node_b)
        if hash_a < hash_b:
            return hash((hash_a, hash_b))

        return hash((hash_b, hash_a))

    edges_dict = {}
    
    for e in edges:
        edge_hash = universal_hash(e.node_a, e.node_b)
        edges_dict[edge_hash] = e
        
    return list(edges_dict.values())

def convert_to_nodes_list(edges):
    nodes = {}
    for e in edges:
        nodes[e.node_a] = e.node_a.copy_without_edges()
        nodes[e.node_b] = e.node_b.copy_without_edges()
        
    for e in edges:
        nodes[e.node_a].add_edge(e)
        nodes[e.node_b].add_edge(e)
        
    return list(nodes.values())

def get_proper_nodes_mask(nodes, shape):
    # removes nodes with degree of 2
    proper_nodes = [node for node in nodes if len(node.edges) != 2]
    print('Number of proper nodes found', len(proper_nodes))
    proper_nodes_mask = np.zeros(shape, dtype=np.uint8)
    for node in proper_nodes:
        for voxel in node['voxels']:
            proper_nodes_mask[tuple(voxel)] = 1
    return proper_nodes_mask, len(proper_nodes)

In [79]:
def remove_redundant_nodes(edges, skeleton_without_bad_edges, thiccness_map):
    iterations_count = 0    
    
    while True:
        edges_without_redundancies = remove_redundant_edges(edges)
        nodes = convert_to_nodes_list(edges_without_redundancies)
        proper_nodes_mask, proper_nodes_count = get_proper_nodes_mask(nodes, skeleton_without_bad_edges.shape)
        if len(nodes) == proper_nodes_count:
            break
        
        edges, graph_bad_edges = construct_graph(
            skeleton_without_bad_edges, proper_nodes_mask, thiccness_map)
        if len(graph_bad_edges) != 0:
            raise Exception("Something went wrong, new bad edges were found ;_;")
        
        iterations_count += 1
        print('Iteration', iterations_count, 'done, nodes removed', len(nodes) - proper_nodes_count)
                
    return nodes, edges_without_redundancies

In [80]:
nodes, new_edges = remove_redundant_nodes(edges, skeleton_without_bad_edges, thiccness_map)
len(new_edges)

Number of proper nodes found 2251
nodes found (regions on nodes mask): 2251
edges found: 2280
Iteration 1 done, nodes removed 49
Number of proper nodes found 2251


2278

In [95]:
def find_tree_root(nodes, root_degree):
    proper_degree_nodes = [node for node in nodes if len(node.edges) == root_degree]
    root_thickness = max(map(lambda node: node['thiccness'], proper_degree_nodes))
    root_candidates = [node for node in proper_degree_nodes if node['thiccness'] == root_thickness]
    return root_candidates

In [97]:
root_candidates = find_tree_root(nodes, 1)
print('number of candidates for root found', len(root_candidates))
root_candidates

number of candidates for root found 1


[Node (395, 79, 279)]

In [118]:
def spherical_kernel(outer_radius, thickness=1, filled=True):    
    outer_sphere = morphology.ball(radius=outer_radius)
    if filled:
        return outer_sphere
    
    thickness = min(thickness, outer_radius)
    
    inner_radius = outer_radius - thickness
    inner_sphere = morphology.ball(radius=inner_radius)
    
    begin = outer_radius - inner_radius
    end = begin + inner_sphere.shape[0]
    outer_sphere[begin:end, begin:end, begin:end] -= inner_sphere
    return outer_sphere

def print_kernels(image, nodes, value):
    image = image.copy()
    max_kernel_radius = int(max([node['thiccness'] for node in nodes]))
    kernels = [spherical_kernel(radius) for radius in range(max_kernel_radius + 1)]
    
    padded_image = np.pad(image, max_kernel_radius)
    kernels_image = np.zeros(padded_image.shape)
    
    for node in nodes:
        x, y, z = (coord + max_kernel_radius for coord in node.coords)
        kernel_radius = int(node['thiccness'])
        kernel = kernels[kernel_radius]
        
        mask_slice = kernels_image[
            x-kernel_radius:x+kernel_radius + 1,
            y-kernel_radius:y+kernel_radius + 1,
            z-kernel_radius:z+kernel_radius + 1
        ]
        
        mask_slice[:] = np.logical_or(mask_slice, kernel)
            
    kernels_image = kernels_image[
        max_kernel_radius:-max_kernel_radius,
        max_kernel_radius:-max_kernel_radius,
        max_kernel_radius:-max_kernel_radius
    ]
    
    image[kernels_image == 1] = value
    return image

def visualise_nodes(image, nodes, value=2):
    nodes_image = print_kernels(image, nodes, value)
    visualize_lsd(nodes_image)

In [119]:
visualise_nodes(skeleton, root_candidates)

In [124]:
all_nodes_skeleton = print_kernels(skeleton, nodes, 2)
all_nodes_with_root_skeleton = print_kernels(all_nodes_skeleton, root_candidates, 3)
visualize_lsd(all_nodes_with_root_skeleton)

### remove cycles

### one more purge after removing cycles

## graph visualization

In [29]:
def draw_graph(edges, shape='auto', lsd=False):
    if shape == 'auto':
        max_x, max_y, max_z = (0, 0, 0)
        for i, edge in enumerate(edges):
            max_x, max_y, max_z = tuple(np.maximum([max_x, max_y, max_z], edge.node_a.coords))
            max_x, max_y, max_z = tuple(np.maximum([max_x, max_y, max_z], edge.node_b.coords))
        shape = (max_x + 1, max_y + 1, max_z + 1)

    graph = np.zeros(shape)

    for i, edge in enumerate(edges):
        graph[line_nd(edge.node_a.coords, edge.node_b.coords)] = i
        
    return graph if lsd else graph > 0

In [30]:
graph = draw_graph(edges, shape = skeleton.shape)

In [13]:
visualize_mask_bin(graph)

In [31]:
def get_main_regions(binary_mask, min_size=10_000, connectivity=3):
    labeled = measure.label(binary_mask, connectivity=connectivity)
    region_props = measure.regionprops(labeled)
    
    main_regions_masks = []
    regions_labels = []
    bounding_boxes = []
    
    for props in region_props:
        if props.area >= min_size:
            main_regions_masks.append(props.filled_image)
            regions_labels.append(props.label)
            bounding_boxes.append(props.bbox)
            
    return main_regions_masks, regions_labels, bounding_boxes

volume = np.pad(load_volume('./data/P12/P12_60um_1333x443x864.raw', scale=0.5), 1)
volume = volume > 70
main_regions = get_main_regions(volume)
mask_main = np.pad(main_regions[0][0].astype(np.uint8), 1)

In [32]:
visualize_addition(graph, mask_main)